### Week 3: Hidden Markov Models
```
- Advanced Machine Learning, Innopolis University 
- Professor: Muhammad Fahim 
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>

```
Lab Plan
    1. Homework 1 Discussion
    2. HMM for POS tagging 
    3. Hidden Markov Models
    4. Manual Calculations
    5. CG rich region identification
```

<hr>

**What's the probability that a random day is sunny?**

**if bob is happy, what's the probability that it's sunny?**

*hint: use Bayes rule.*

In [0]:
import numpy as np
import torch
states = ('Sunny', 'Rainy')
observations = ('happy', 'grumpy')
pi = np.array([2./3., 1./3.])  #initial probability 
A = np.array([[7./9., 2./9.],[0.4, 0.6]]) #Transmission probability 
B = np.array([[0.8, 0.2],[0.4, 0.6]]) #Emission probability
bob_says = np.array([0,0,1,1,1,0])

def forward(obs_seq, pi, A, B):
    T = len(obs_seq)
    N = A.shape[0]
    alpha = np.zeros((T, N))
    alpha[0] = pi*B[:,obs_seq[0]]
    for t in range(1, T):
        alpha[t] = np.inner(alpha[t-1],A) * B[:, obs_seq[t]]
    return alpha

def likelihood(alpha):
    # returns log P(Y  \mid  model)
    # using the forward part of the forward-backward algorithm
    return  alpha[-1].sum()  

alpha = forward(bob_says, pi, A, B)
print(alpha)

print(likelihood(alpha))

[[0.53333333 0.13333333]
 [0.35555556 0.11733333]
 [0.06052346 0.12757333]
 [0.01508469 0.06045203]
 [0.00503326 0.02538306]
 [0.00764435 0.00689726]]
0.014541607035957256


### **Viterbi Algorithm**


In [0]:
from numpy import random
# Transition Probabilities
p_ss = 7./9.
p_sr = 2./9.
p_rs = 0.4
p_rr = 0.6

# Initial Probabilities
p_s = 2/3
p_r = 1/3

# Emission Probabilities
p_sh = 0.8
p_sg = 0.2
p_rh = 0.4
p_rg = 0.6

moods = ['H', 'H', 'G', 'G', 'G', 'H']
probabilities = []
weather = []

if moods[0] == 'H':
    probabilities.append((p_s*p_sh, p_r*p_rh))
else:
    probabilities.append((p_s*p_sg, p_r*p_rg))

for i in range(1,len(moods)):
    yesterday_sunny, yesterday_rainy = probabilities[-1]
    if moods[i] == 'H':
        today_sunny = max(yesterday_sunny*p_ss*p_sh, yesterday_rainy*p_rs*p_sh)
        today_rainy = max(yesterday_sunny*p_sr*p_rh, yesterday_rainy*p_rr*p_rh)
        probabilities.append((today_sunny, today_rainy))
    else:
        today_sunny = max(yesterday_sunny*p_ss*p_sg, yesterday_rainy*p_rs*p_sg)
        today_rainy = max(yesterday_sunny*p_sr*p_rg, yesterday_rainy*p_rr*p_rg)
        probabilities.append((today_sunny, today_rainy))

for p in probabilities:
    if p[0] > p[1]:
        weather.append('S')
    else:
        weather.append('R')
        
weather

['S', 'S', 'S', 'R', 'R', 'S']

In [0]:
probabilities

[(0.5333333333333333, 0.13333333333333333),
 (0.3413333333333334, 0.04266666666666667),
 (0.05461333333333335, 0.04096000000000001),
 (0.008738133333333337, 0.014745600000000001),
 (0.0013981013333333341, 0.005308416),
 (0.00169869312, 0.00127401984)]

### Is there any Python package for all these computation? 

![](https://cdn.pixabay.com/photo/2018/03/25/11/43/pomegranate-3259161_960_720.jpg)

Hidden Markov models (HMMs) are the flagship of the pomegranate package in that they have the most features of all of the models and that they were the first algorithm implemented.

## CG rich region identification


Hidden Markov models are a form of structured prediction method that are popular for tagging all elements in a sequence with some "hidden" state. They can be thought of as extensions of Markov chains where, instead of the probability of the next observation being dependant on the current observation, the probability of the next hidden state is dependant on the current hidden state, and the next observation is derived from that hidden state. An example of this can be part of speech tagging, where the observations are words and the hidden states are parts of speech. Each word gets tagged with a part of speech, but dynamic programming is utilized to search through all potential word-tag combinations to identify the best set of tags across the entire sentence.

Another perspective of HMMs is that they are an extension on mixture models that includes a transition matrix. Conceptually, a mixture model has a set of "hidden" states---the mixture components---and one can calculate the probability that each sample belongs to each component. This approach treats each observations independently. However, like in the part-of-speech example we know that an adjective typically is followed by a noun, and so position in the sequence matters. A HMM adds a transition matrix between the hidden states to incorporate this information across the sequence, allowing for higher probabilities of transitioning from the "adjective" hidden state to a noun or verb.

pomegranate implements HMMs in a flexible manner that goes beyond what other packages allow. Let's see some examples.

In [3]:
!pip3 install pomegranate

     |████████████████████████████████| 20.9 MB 23.9 MB/s eta 0:00:01     |████████████████████████████▉   | 18.9 MB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 24.2 MB/s eta 0:00:01
ERROR: tensorflow 2.3.2 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.20.2 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set_style('whitegrid')
import numpy

from pomegranate import *

numpy.random.seed(0)
numpy.set_printoptions(suppress=True)


**CG rich region identification example**
Lets take the simplified example of CG island detection on a sequence of DNA. DNA is made up of the four canonical nucleotides, abbreviated 'A', 'C', 'G', and 'T'. We can say that regions of the genome that are enriched for nucleotides 'C' and 'G' are 'CG islands', which is a simplification of the real biological concept but sufficient for our example. The issue with identifying these regions is that they are not exclusively made up of the nucleotides 'C' and 'G', but have some 'A's and 'T's scatted amongst them. A simple model that looked for long stretches of C's and G's would not perform well, because it would miss most of the real regions.

We can start off by building the model. Because HMMs involve the transition matrix, which is often represented using a graph over the hidden states, building them requires a few more steps that a simple distribution or the mixture model. Our simple model will be composed of two distributions. One distribution wil be a uniform distribution across all four characters and one will have a preference for the nucleotides C and G, while still allowing the nucleotides A and T to be present.

In [5]:
d1 = DiscreteDistribution({'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25})
d2 = DiscreteDistribution({'A': 0.10, 'C': 0.40, 'G': 0.40, 'T': 0.10})

# For the HMM we have to first define states, which are a pair of a distribution and a name.
s1 = State(d1, name='background')
s2 = State(d2, name='CG island')

In [6]:
# Now we define the HMM and pass in the states.
model = HiddenMarkovModel()
model.add_states(s1, s2)

Then we have to define the transition matrix, which is the probability of going from one hidden state to the next hidden state. In some cases, like this one, there are high self-loop probabilities, indicating that it's likely that one will stay in the same hidden state from one observation to the next in the sequence. Other cases have a lower probability of staying in the same state, like the part of speech tagger. A part of the transition matrix is the start probabilities, which is the probability of starting in each of the hidden states. Because we create these transitions one at a time, they are very amenable to sparse transition matrices, where it is impossible to transition from one hidden state to the next.

In [7]:
model.add_transition(model.start, s1, 0.5)
model.add_transition(model.start, s2, 0.5)
model.add_transition(s1, s1, 0.9)
model.add_transition(s1, s2, 0.1)
model.add_transition(s2, s1, 0.1)
model.add_transition(s2, s2, 0.9)

In [8]:
model.bake()

In [9]:
seq = numpy.array(list('CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC'))

hmm_predictions = model.predict(seq)

print("sequence: {}".format(''.join(seq)))
print("hmm pred: {}".format(''.join(map( str, hmm_predictions))))

sequence: CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC
hmm pred: 111111111111111000000000000000011111111111111110000


Note that all we did was add a transition from `s1` to `model.end` with some low probability. This probability doesn't have to be high if there's only a single transition there, because there's no other possible way of getting to the end state.

In [10]:
model = HiddenMarkovModel()
model.add_states(s1, s2)
model.add_transition(model.start, s1, 0.5)
model.add_transition(model.start, s2, 0.5)
model.add_transition(s1, s1, 0.89 )
model.add_transition(s1, s2, 0.10 )
model.add_transition(s1, model.end, 0.01)
model.add_transition(s2, s1, 0.1 )
model.add_transition(s2, s2, 0.9)
model.bake()

seq = numpy.array(list('CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC'))

hmm_predictions = model.predict(seq)

print("sequence: {}".format(''.join(seq)))
print("hmm pred: {}".format(''.join(map( str, hmm_predictions))))

sequence: CGACTACTGACTACTCGCCGACGCGACTGCCGTCTATACTGCGCATACGGC
hmm pred: 111111111111111000000000000000011111111111111111111
